In [ ]:
import pandas as pd
import re
import sqlite3 as sql
from IPython.display import Markdown, display
from ipywidgets import  widgets,Layout
from PIL import Image
def displayRecord(change):    
    value=change['new']
    df=searchGua(change)    
    text_box.value=str(len)
    # print( f'当前数据编号: {value}')
    content=df.loc[value,'guaContent']
    content=content.replace("\n","\n\n")
    text_box.value=content
    re1=r'images.+jpg|images.+.jpeg|images.+png'
    images=re.findall(re1,content)
    img_dropdown.options=images
    img_dropdown.index=1
    displayImage(change)

def displayImages(change):
    content=""
    for item in img_dropdown.options:
        content=content+"![]("+item+")"   
    # content=img_dropdown.value
    # content="![]("+content+")"    
    with output2:
        output2.clear_output()
        display(Markdown(content))
        # display(Image.open(content))

def searchGua(change):
    gua_name_str=gua_name_text.value
    database="Guas.db"
    connection=sql.connect(database)
    # gua_name_str="泰静卦"
    query=f'select rowid, guaDate, guaName,stockName,substr(guaSubject,1,20),guaContent from StockGuas  where guaName like "{gua_name_str}"   '
    df=pd.read_sql_query(query,connection)
    df_display=df[['rowid', 'guaDate','guaName', 'stockName','substr(guaSubject,1,20)']]
    output1.clear_output( )
    output2.clear_output()
    text_box.value=""
    length=df.shape[0]
    slider.max=length
    with output1:
        print(df_display)
    connection.close()
    return df

slider = widgets.IntSlider(value=0, min=0, max=20, description='查询编号')
gua_name_text = widgets.Text(description='卦名:', layout=Layout(width='150px', height='20px'))
monthday_text = widgets.Text(description='日月:', layout=Layout(width='150px', height='20px'))
xunkong_text = widgets.Text(description='旬空:', layout=Layout(width='150px', height='20px'))
button_search = widgets.Button(description='搜索', layout=Layout(width='100px', height='20px'))
output1=widgets.Output(layout=Layout(width='80%', height='200px'))
output2=widgets.Output(layout=Layout(width='80%', height='200px'))

img_dropdown = widgets.Dropdown(discription='选择图片',layout=Layout(width='300px', height='20px'))
text_box = widgets.Textarea(value='',placeholder='markdown内容', layout=Layout(width='50%', height='400px'))

search_bar=widgets.HBox([button_search,gua_name_text,monthday_text,xunkong_text])
search_box=widgets.VBox([search_bar,output1])
image_box=widgets.VBox([img_dropdown,output2])
detail_box=widgets.HBox([text_box,image_box])
main_layout=widgets.VBox([search_box,slider,detail_box])
button_search.on_click(searchGua)
slider.observe(displayRecord,names='value')
img_dropdown.observe(displayImages,names='value')
# widgets.interact(displayRecord, value=slider)
# button_disp.on_click(displayRecord)
display(main_layout)
                  

In [2]:
# 最简单搜索程序，按卦名+干支+旬空，搜索相关记录到df
#下一个程序则显示各记录细节
import sqlite3 as sql
import pandas as pd

gua_name_str=input("输入卦名:")
gangzhi_str=input("输入日月干支:")
xunkong_str=input("输入旬空:")
database="Guas.db"
connection=sql.connect(database)
query=f'select rowid, guaDate, guaName, stockName, substr(guaSubject,1,20) as Subject, guaContent from StockGuas  where guaName like "{gua_name_str}"   '
orderby= '  order by cast( substr(guaDate,6,2) as integer),guaDate '
sub2=""
sub3=""
if gangzhi_str!="":
    sub2=f' and gangZhi like "%{gangzhi_str}%"  '
if xunkong_str!="":
    sub3=f' and xunKong like "%{xunkong_str}%"  '
query=query+sub2+sub3+orderby
df=pd.read_sql_query(query,connection)
df_disp=df[['rowid', 'guaDate','guaName','stockName','Subject']]
df_disp=df_disp.rename(columns = {'rowid':'记录号' ,'guaDate':'日期','guaName':'卦名','stockName':'股票','Subject':'预测主题'})
df_disp


输入卦名: 大壮之夬
输入日月干支: 
输入旬空: 


,记录号,日期,卦名,股票,预测主题
0,15846,NULL,大壮之夬,增删,大壮之夬，高岛易断.md
1,18708,2014-01-30,大壮之夬,600158,占事：600158中体产业年卦排卦：元亨
2,18811,2015-01-06,大壮之夬,600104,大壮之夬，测上汽集团未来一个月。.md
3,7990,2016-01-26,大壮之夬,600758,丙申年股票实战卦总卦贴
4,6676,2017-01-10,大壮之夬,上证,测2017.1.9-1.13上证哪日是顶
5,19720,2023-01-31,大壮之夬,301132,个股年卦：2023年卦满坤科技半年卦。大
6,12203,2009-02-20,大壮之夬,大盘,占事:今日股市公历时间：2009年2月2
7,16273,2011-02-27,大壮之夬,NULL,经典：大壮之夬，子化进。标准大涨卦。青岛
8,10065,2013-02-14,大壮之夬,大盘,癸巳年上证指数预测
9,2658,2021-02-06,大壮之夬,大盘,3：是4吗？


In [3]:
df_disp.head(3)

,记录号,日期,卦名,股票,预测主题
0,15846,NULL,大壮之夬,增删,大壮之夬，高岛易断.md
1,18708,2014-01-30,大壮之夬,600158,占事：600158中体产业年卦排卦：元亨
2,18811,2015-01-06,大壮之夬,600104,大壮之夬，测上汽集团未来一个月。.md


In [41]:
import ipywidgets as widgets
from IPython.display import HTML,display,Markdown
import sixyao

def paipan(change):
    output.clear_output()
    gua=sixyao.Zhugua()
    day1=gua_date_text.value
    namestr=gua_name_text.value
    if day1=="" or namestr=="":
        return  "非法动作-无日期或卦名"
    if '之' in namestr:
        split_string=namestr.split('之')
        name1 = split_string[0].strip() 
        name2 = split_string[1].strip() 
    else:
        name1=namestr.strip("静卦")
        name2=namestr.strip("静卦")
    gzstring=gua.setDate(day1)
    gua.makeGuaByName(name1,name2)
    #outname,guacont=gua.displayDoubleGuaText()
    outGuaName,guacont=gua.displayDoubleGuaText(has_yaochi=False)
    temp=text_box.value+guacont
    text_box.value=temp
    
def record_show(change):
    value=change['new']
    # print( f'当前数据编号: {value}')
    gua_name_text.value=df.loc[value,'guaName']
    gua_date_text.value=df.loc[value,'guaDate']
    stock_name_text.value=df.loc[value,'stockName']
    
    text_box.value=df.loc[value,'guaContent']
    # content=content.replace("\n","\n\n")
    re1=r'images.+jpg|images.+.jpeg|images.+png'
    content=text_box.value
    image_show(change)
    # with output1:
    #     output1.clear_output()
    #     display(Markdown(content))
        
def image_show(change):
    re1=r'images.+jpg|images.+.jpeg|images.+png'
    content=text_box.value
    images=re.findall(re1,content)
    pics_content=""
    for item in images:
        pics_content=pics_content+"![]("+item+")" 
    with output:
        output.clear_output()
        display(Markdown(pics_content))
        # for item in images:
        #     print(item)  
length=df_disp.shape[0]
print("df length=", length)
slider1 = widgets.IntSlider(value=0, min=0, max=length-1, description='卦例')
gua_name_text = widgets.Text(description='卦名:', layout=Layout(width='150px', height='20px'))
gua_date_text = widgets.Text(description='日期:', layout=Layout(width='150px', height='20px'))
stock_name_text = widgets.Text(description='股票:', layout=Layout(width='150px', height='20px'))
button_sixyao = widgets.Button(description='生成六爻', layout=Layout(width='100px', height='20px'))
button_draw15 = widgets.Button(description='绘图15天', layout=Layout(width='100px', height='20px'))
button_draw35 = widgets.Button(description='绘图35天', layout=Layout(width='100px', height='20px'))
button_draw365 = widgets.Button(description='绘图一年', layout=Layout(width='100px', height='20px'))
button_save_text = widgets.Button(description='保存修改', layout=Layout(width='100px', height='20px'))
text_box = widgets.Textarea(value='',placeholder='markdown内容', layout=Layout(width='60%', height='400px'))
# text_box = widgets.Textarea(value='',placeholder='markdown内容', layout=Layout(width='50%', height='400px'))
output=widgets.Output(layout=Layout(width='40%', height='500px'))
top_layout=widgets.HBox([slider1,button_sixyao ,button_save_text, gua_name_text,gua_date_text,stock_name_text , button_draw15,button_draw35,button_draw365])
main_layout=widgets.HBox([text_box,output])
# output2=widgets.Output(layout=Layout(width='40%', height='500px'))
slider1.observe(record_show, names='value')
slider2.observe(image_show,names='value')
button_sixyao.on_click(paipan)
# widgets.interact(format_slider, value=slider1)
# widgets.interact(image_show, value=slider2)
display(top_layout,main_layout)

# df_disp

df length= 51
